# 3D analysis

This tutorial shows how to run a 3D map-based analysis using some 1DC observations of the Galactic center region with CTA. For this tutorial the $CTADATA environment variable will be used (see https://forge.in2p3.fr/projects/data-challenge-1-dc-1/wiki/Getting_data). More information about CTA DC1 can be found in https://forge.in2p3.fr/projects/data-challenge-1-dc-1/wiki as well as in "cta_1dc_introduction" and "cta_data_analysis" tutorial notebooks.

(note: part of this notebook can be run using $GAMMAPY_DATA variable instead of CTADATA, but not the galactic diffuse emission part yet)

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import os
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.extern.pathlib import Path
from gammapy.data import DataStore
from gammapy.irf import EnergyDispersion
from gammapy.maps import WcsGeom, MapAxis, Map, WcsNDMap
from gammapy.cube import MapMaker, MapEvaluator, PSFKernel, MapFit
from gammapy.cube.models import SkyModel,SkyDiffuseCube
from gammapy.spectrum.models import PowerLaw, ExponentialCutoffPowerLaw
from gammapy.image.models import SkyGaussian, SkyPointSource
from regions import CircleSkyRegion

In [ ]:
path = os.path.expandvars("$CTADATA") 

if not os.path.exists(path):
    raise Exception("$CTADATA repository not found!")
else:
    print("Great! your setup is correct.")
    !gammapy info --no-envvar --no-system

## Prepare modeling input data

### Prepare input maps

We first use the `DataStore` object to access the CTA observations and, after applying some selection cuts, we pass the selected observations ID's to the `.obs_list()` method:

In [ ]:
# Define which data to use and print some information
data_store = DataStore.from_dir('$CTADATA/index/gps')
data_store.info()
print('Total observation time (hours): ', data_store.obs_table['ONTIME'].sum() / 3600)
print('Observation table: ', data_store.obs_table.colnames)
print('HDU table: ', data_store.hdu_table.colnames)

In [ ]:
# Select some observations from these dataset
table = data_store.obs_table
pos_obs = SkyCoord(table['GLON_PNT'], table['GLAT_PNT'], frame='galactic', unit='deg')
pos_target = SkyCoord(0, 0, frame='galactic', unit='deg')
offset = pos_target.separation(pos_obs).deg
mask = (0 < offset) & (offset < 3)
table = table[mask]

In [ ]:
# Print total number and ID's of observation list
print("{} observations selected:".format(len(table)))
table["OBS_ID"]
obs_id = table["OBS_ID"].tolist()
obs_list = data_store.obs_list(obs_id)
print(obs_id)

Now we define a reference geometry for our analysis, We choose a WCS based gemoetry with a binsize of 0.02 deg and also define an energy axis: 

In [ ]:
energy_axis = MapAxis.from_edges(
    np.logspace(-1., 1., 10), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(10, 8),
    coordsys="GAL",
    proj="CAR",
    axes=[energy_axis],
)

The `MapMaker` object is initialized with this reference geometry and a field of view cut of 4 deg:

In [ ]:
%%time
maker = MapMaker(geom, offset_max=4. * u.deg)
maps = maker.run(obs_list)

The maps are prepared by calling the `.run()` method and passing the observation list `obs_list`. The `.run()` method returns a Python `dict` containing a `counts`, `background` and `exposure` map:

In [ ]:
print(maps)

This is what the summed counts image looks like:

In [ ]:
counts = maps["counts"].sum_over_axes()
counts.smooth(width=0.1 * u.deg).plot(stretch="sqrt", add_cbar=True, vmax = 90);

And the background image:

In [ ]:
background = maps["background"].sum_over_axes()
background.smooth(width=0.1 * u.deg).plot(stretch="sqrt", add_cbar=True, vmax=90);

And also the exposure image:

In [ ]:
exposure = maps["exposure"].sum_over_axes()
exposure.smooth(width=0.1 * u.deg).plot(stretch="sqrt", add_cbar=True);

We can also compute an excess image just with  a few lines of code:

In [ ]:
excess = Map.from_geom(geom.to_image())
excess.data = counts.data - background.data
excess.smooth(5).plot(stretch="sqrt", add_cbar=True);

For a more realistic excess plot we can also take into account the diffuse galactic emission:

In [ ]:
#A cutoff can speed things up here!
diffuse_gal = Map.read(
    "$CTADATA/models/cube_iem.fits.gz"
)

In [ ]:
print("Diffuse image: ",diffuse_gal.geom)
print("counts: ",maps["counts"].geom)

We see that the geometry of the images is completely different, so we need to apply our geometric configuration to the diffuse emission file:

In [ ]:
coord = maps["counts"].geom.get_coord()

data = diffuse_gal.interp_by_coord(
    {
        "skycoord": coord.skycoord,
        "energy": coord["energy"]
        * maps["counts"].geom.get_axis_by_name("energy").unit,
    },
    interp=3,
)
diffuse_galactic = WcsNDMap(
    maps['counts'].geom, data
)
print("Before: ",diffuse_gal.geom)
print("Now (same as maps): ", diffuse_galactic.geom)

In [ ]:
#diffuse_galactic.slice_by_idx({"energy": 0}).plot(add_cbar=True);
diffuse = diffuse_galactic.sum_over_axes()
diffuse.smooth(5).plot(stretch="sqrt", add_cbar=True);
print(diffuse)

We now multiply the exposure for this diffuse emission to substract it from the counts along with the background.

In [ ]:
combination = Map.from_geom(geom.to_image())
combination.data = diffuse.data*exposure.data
combination.smooth(5).plot(stretch="sqrt", add_cbar=True);

We can plot then the excess image substracting now the effect of the diffuse galactic emission.

In [ ]:
excess2 = Map.from_geom(geom.to_image())
excess2.data = counts.data - background.data - combination.data

fig, axs = plt.subplots(1,2, figsize=(15, 5))

axs[0].set_title("With diffuse emission substraction")
axs[1].set_title("Without diffuse emission substraction")
excess2.smooth(5).plot(cmap="coolwarm", vmin = -10, vmax = 10, add_cbar=True, ax=axs[0]);
excess.smooth(5).plot(cmap="coolwarm", vmin = -10, vmax = 10, add_cbar=True, ax=axs[1]);

### Prepare IRFs

To estimate the mean PSF across all observations at a given source position `src_pos`, we use the `make_mean_psf()` method:

In [ ]:
from gammapy.irf import make_mean_psf

# mean PSF
src_pos = SkyCoord(0, 0, unit="deg", frame="galactic")
table_psf = make_mean_psf(obs_list, src_pos)

# PSF kernel used for the model convolution
psf_kernel = PSFKernel.from_table_psf(table_psf, geom, max_radius="0.3 deg")

To estimate the mean energy dispersion across all observations at a given source position `src_pos`, we use the `make_mean_edisp()` method:

In [ ]:
from gammapy.irf import make_mean_edisp

# define energy grid
energy = energy_axis.edges * energy_axis.unit

# mean edisp
edisp = make_mean_edisp(
    obs_list, position=src_pos, e_true=energy, e_reco=energy
)

### Save maps and IRFs to disk

It is common to run the preparation step independent of the likelihood fit, because often the preparation of maps, PSF and energy dispersion is slow if you have a lot of data. We first create a folder:

In [ ]:
path = Path("TEST_analysis_3d")
path.mkdir(exist_ok=True)

And then write the maps and IRFs to disk by calling the dedicated `.write()` methods:

In [ ]:
# write maps
maps["counts"].write(str(path / "counts.fits"), overwrite=True)
maps["background"].write(str(path / "background.fits"), overwrite=True)
maps["exposure"].write(str(path / "exposure.fits"), overwrite=True)

# write IRFs
psf_kernel.write(str(path / "psf.fits"), overwrite=True)
edisp.write(str(path / "edisp.fits"), overwrite=True)

## Likelihood fit

### Reading maps and IRFs
As first step we read in the maps and IRFs that we have saved to disk again:

In [ ]:
# read maps
maps = {
    "counts": Map.read(str(path / "counts.fits")),
    "background": Map.read(str(path / "background.fits")),
    "exposure": Map.read(str(path / "exposure.fits")),
}

# read IRFs
psf_kernel = PSFKernel.read(str(path / "psf.fits"))
edisp = EnergyDispersion.read(str(path / "edisp.fits"))

Let's cut out only part of the maps, so that we the fitting step does not take so long (we go from left to right one):

In [ ]:
cmaps = {
    name: m.cutout(SkyCoord(0, 0, unit="deg", frame="galactic"), 2 * u.deg)
    for name, m in maps.items()
}
cmaps["counts"].sum_over_axes().plot(stretch="sqrt");

Insted of the complete one, which was:

In [ ]:
counts.plot(stretch="sqrt");

### Fit mask

To select a certain spatial region and/or energy range for the fit we can create a fit mask:

In [ ]:
mask = Map.from_geom(cmaps["counts"].geom)

region = CircleSkyRegion(center=src_pos, radius=0.6 * u.deg)
mask.data = mask.geom.region_mask([region])

mask.get_image_by_idx((0,)).plot();

In addition we also exclude the range below 0.3 TeV for the fit:

In [ ]:
coords = mask.geom.get_coord()
mask.data &= coords["energy"] > 0.3

### Model fit

No we are ready for the actual likelihood fit. We first define the model as a combination of a point source with an exponential cutt off power law:

In [ ]:
spatial_model = SkyPointSource(lon_0="0.01 deg", lat_0="0.01 deg")
spectral_model = ExponentialCutoffPowerLaw(
    index=2 * u.Unit(''),
    amplitude=1e-12 * u.Unit('cm-2 s-1 TeV-1'),
    reference=1. * u.TeV,
    lambda_=1 / u.TeV
)
model = SkyModel(spatial_model=spatial_model, spectral_model=spectral_model)

Now we set up the `MapFit` object by passing the prepared maps, IRFs as well as the model:

In [ ]:
fit = MapFit(
    model=model,
    counts=cmaps["counts"],
    exposure=cmaps["exposure"],
    background=cmaps["background"],
    mask=mask,
    psf=psf_kernel,
    edisp=edisp,
)

No we run the model fit:

In [ ]:
%%time
result = fit.run(optimize_opts={"print_level": 1})

In [ ]:
print(result.model)

### Check model fit

We check the model fit by computing a residual image. For this we first get the number of predicted counts from the fit evaluator:

In [ ]:
npred = fit.evaluator.compute_npred()

And compute a residual image:

In [ ]:
residual = Map.from_geom(cmaps["counts"].geom)
residual.data = cmaps["counts"].data - npred.data

In [ ]:
residual.sum_over_axes().smooth(width=0.05 * u.deg).plot(
    cmap="coolwarm", vmin = -40, vmax=40, add_cbar=True
);

Apparently our model should be improved by adding a component for diffuse Galactic emission and at least one second point
source (see exercises at the end of the notebook).

We can also plot the best fit spectrum:

In [ ]:
spec = result.model.spectral_model
energy_range = [0.3, 10] * u.TeV
spec.plot(energy_range=energy_range, energy_power=2)
ax = spec.plot_error(energy_range=energy_range, energy_power=2)

This result can be compared with the model used for the 1DC:

In [ ]:
dc1_model = ExponentialCutoffPowerLaw(
    index = 2.14 * u.Unit(''),
    amplitude = 2.55e-12 * u.Unit('cm-2 s-1 TeV-1'),
    reference= 1. * u.TeV,
    lambda_= 0.0934 / u.TeV
)
dc1_model.parameters.to_table()

In [ ]:
result.model.parameters.to_table()

In [ ]:
spec = result.model.spectral_model
energy_range = [0.3, 10] * u.TeV
spec.plot(energy_range=energy_range, energy_power=2)
ax = spec.plot_error(energy_range=energy_range, energy_power=2)
ax = dc1_model.plot(energy_range, energy_power=2, color='red')
ax.legend(['Spectral Fit', 'DC1 model'])

We can see this discrepancy between the fit and the DC1 model due to the diffuse emission... so let's improve this.

### Adding galactic diffuse emission to model

We use both models at the same time, our diffuse model (taken from 1DC idem file) and our previous model for the central source (note that we are not constraining the fit with any mask this time).

In [ ]:
diffuse_model = SkyDiffuseCube.read("$CTADATA/models/cube_iem.fits.gz")

In [ ]:
combined_fit = MapFit(
    model=diffuse_model+model,
    counts=cmaps["counts"],
    exposure=cmaps["exposure"],
    background=cmaps["background"],
    psf=psf_kernel
)

In [ ]:
%%time
result_combined=combined_fit.run()

In [ ]:
print(result_combined.model)

As we can see we have now two components in our model, and we can access them separately.

In [ ]:
#Checking normalization value (the closer to 1 the better)
print("FIRST MODEL (SkyDiffuseCube): ",result_combined.model.model1.parameters)
print("SECOND MODEL (SkyModel): ",result_combined.model.model2.parameters)

Obtaining a normalization parameter of 0.77, which is not bad... We can now plot the residual image considering this improved model.

In [ ]:
npred_combined = combined_fit.evaluator.compute_npred()

In [ ]:
import matplotlib.cm as cm
residual2 = Map.from_geom(cmaps["counts"].geom)
residual2.data = cmaps["counts"].data - npred_combined.data
residual2.sum_over_axes().smooth(width=0.05 * u.deg).plot(
    cmap = cm.jet, vmin = -30, vmax=30, add_cbar=True
);

Just as a comparison, we can plot our previous residual map (left) and the new one (right) with the same scale:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 5))

axs[0].set_title("Without diffuse emission substraction")
axs[1].set_title("With diffuse emission substraction")
residual.sum_over_axes().smooth(width=0.05 * u.deg).plot(
    cmap="coolwarm", vmin = -40, vmax=40, add_cbar=True, ax=axs[0]
);
residual2.sum_over_axes().smooth(width=0.05 * u.deg).plot(
    cmap="coolwarm", vmin = -40, vmax=40, add_cbar=True, ax=axs[1]
);

Finally we can compare again our model (including now the diffuse emission) with the one used for the DC1

In [ ]:
spec2 = result_combined.model.model2.spectral_model
spec2.plot(energy_range=energy_range, energy_power=2)
ax = dc1_model.plot(energy_range, energy_power=2, color='red')
ax.legend(['Spectral Fit', 'DC1 model'])

Results seems to be better (but not perfect yet). As an example, let's compare flux values at 1TeV for both models:

In [ ]:
spec2.evaluate(energy=1*u.TeV, index = 2.079 * u.Unit(''), amplitude=2.662e-12 * u.Unit('cm-2 s-1 TeV-1'), 
                   reference=1. * u.TeV, lambda_=1.149e-01 / u.TeV)

In [ ]:
dc1_model.evaluate(energy=1*u.TeV, index = 2.14 * u.Unit(''), amplitude=2.55e-12 * u.Unit('cm-2 s-1 TeV-1'), 
                   reference=1. * u.TeV, lambda_=0.0934 / u.TeV)

Next step to improve our model even more would be getting rid of the other bright source (G0.9+0.1).

## Exercises

* Analyse the second source in the field of view: G0.9+0.1 and add it to the combined model.
* Run the model fit with energy dispersion (pass edisp to MapFit with different true and reco energy bins).